In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11338
2,Huelva,Confirmados PDIA 14 días,613
3,Huelva,Tasa PDIA 14 días,"120,02122398153278"
4,Huelva,Confirmados PDIA 7 días,244
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,8
629,Municipio de Huelva sin especificar,Total Confirmados,134
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11338.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3225.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1651 personas en los últimos 7 días 

Un positivo PCR cada 718 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11338.0,244.0,613.0,510743.0,47.773538,120.021224,19.0
Huelva-Costa,6431.0,164.0,417.0,288115.0,56.921715,144.733874,15.0
Huelva (capital),3225.0,87.0,200.0,143663.0,60.558390,139.214690,8.0
Punta Umbría,241.0,6.0,9.0,15242.0,39.364913,59.047369,4.0
Sierra de Huelva-Andévalo Central,1302.0,16.0,19.0,67571.0,23.678797,28.118572,3.0
Cerro de Andévalo (El),47.0,9.0,10.0,2364.0,380.710660,423.011844,2.0
Condado-Campiña,3472.0,56.0,167.0,155057.0,36.115751,107.702329,2.0
Almonte,450.0,6.0,13.0,24191.0,24.802613,53.738994,1.0
Isla Cristina,469.0,19.0,59.0,21264.0,89.352897,277.464259,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),47.0,9.0,10.0,2364.0,380.710660,423.011844,2.0
Villablanca,67.0,10.0,12.0,2848.0,351.123596,421.348315,1.0
Lucena del Puerto,106.0,2.0,12.0,3371.0,59.329576,355.977455,0.0
Manzanilla,52.0,7.0,7.0,2135.0,327.868852,327.868852,0.0
Palma del Condado (La),555.0,13.0,35.0,10761.0,120.806616,325.248583,0.0
Bonares,67.0,2.0,19.0,6058.0,33.014196,313.634863,0.0
Isla Cristina,469.0,19.0,59.0,21264.0,89.352897,277.464259,1.0
Palos de la Frontera,261.0,10.0,31.0,11289.0,88.581805,274.603596,0.0
Cartaya,430.0,13.0,50.0,19974.0,65.084610,250.325423,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,67.0,2.0,19.0,6058.0,33.014196,313.634863,0.0,0.105263
Lucena del Puerto,106.0,2.0,12.0,3371.0,59.329576,355.977455,0.0,0.166667
Rociana del Condado,121.0,1.0,5.0,7855.0,12.730745,63.653724,0.0,0.200000
Aljaraque,443.0,5.0,24.0,21260.0,23.518344,112.888053,0.0,0.208333
Cartaya,430.0,13.0,50.0,19974.0,65.084610,250.325423,0.0,0.260000
Isla Cristina,469.0,19.0,59.0,21264.0,89.352897,277.464259,1.0,0.322034
Palos de la Frontera,261.0,10.0,31.0,11289.0,88.581805,274.603596,0.0,0.322581
Condado-Campiña,3472.0,56.0,167.0,155057.0,36.115751,107.702329,2.0,0.335329
Palma del Condado (La),555.0,13.0,35.0,10761.0,120.806616,325.248583,0.0,0.371429
